# Preview Evaluation - Perplexity

Thia is a notebook to make a preview comparison of the generations when using Perplexity as a metric for the Datamodels collections.  
Important to notice, this notebook is considering onle ONE generation of 50 samples.

In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from utils.calculate_metric import calculate_agg_metric

## Calculate Metrics and save them

In [3]:
datamodels_path  = "../dmrc_pipelines/7270"
questions_path = "../../datamodels_training_window_size/experiments_7270/questions.feather"

In [4]:
calculate_agg_metric(
    metrics=["rouge_l"],
    generation_path=f"{datamodels_path}/generations/datamodels_generations.json",
    reference_path=questions_path,
    saving_path="preview.feather"
)

## Load Data

In [7]:
df = pl.read_ipc("preview.feather")
df.select("mean").mean()

mean
f64
0.009194


In [11]:
import json


rag_retrieval = json.load(open(f"{datamodels_path}/retrieval/rag_retrieval_indexes.json"))
per_datamodels_retrieval = json.load(open(f"{datamodels_path}/retrieval/7270_indexes.json"))
rou_datamodels_retrieval = json.load(open("rouge_indexes.json"))

In [25]:
import numpy as np


dicts = [rag_retrieval, per_datamodels_retrieval, rou_datamodels_retrieval]
processed_dicts = []
for d in dicts:
    # Get values in order and convert to numpy array
    array_50x100 = np.array([d[str(key)] for key in range(50)])
    # Take first 16 elements
    array_50x16 = array_50x100[:, 34:50]
    # Convert back to list if needed (otherwise keep as numpy array)
    processed_dicts.append(array_50x16.tolist())

In [26]:
from polars import count


def count_matches_no_numpy(list_a, list_b):
    matches = 0
    for key in range(50):
        for i in range(16):
            if list_a[key][i] == list_b[key][i]:
                matches += 1
    return matches

rag, per, rouge = processed_dicts

print(count_matches_no_numpy(rag, per))
print(count_matches_no_numpy(rag, rouge))
print(count_matches_no_numpy(per, rouge))

7
0
0
